### ¡Bienvenidos a la Semana 6 Día 3!

Vamos a experimentar con muchos más servidores MCP

In [28]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
import os
from IPython.display import Markdown, display
from datetime import datetime
load_dotenv(override=True)

True

### El primer tipo de servidor MCP: se ejecuta localmente, todo local

Aquí hay uno realmente interesante: una memoria basada en grafo de conocimiento.

Es un almacén de memoria persistente de entidades, observaciones sobre ellas y relaciones entre ellas.

https://github.com/modelcontextprotocol/servers/tree/main/src/memory


In [2]:
params = {"command": "npx","args": ["-y", "mcp-memory-libsql"],"env": {"LIBSQL_URL": "file:./memory/juangabriel.db"}}

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()

mcp_tools

[Tool(name='create_entities', description='Create new entities with observations and optional embeddings', inputSchema={'type': 'object', 'properties': {'entities': {'type': 'array', 'items': {'type': 'object', 'properties': {'name': {'type': 'string'}, 'entityType': {'type': 'string'}, 'observations': {'type': 'array', 'items': {'type': 'string'}}, 'embedding': {'type': 'array', 'items': {'type': 'number'}, 'description': 'Optional vector embedding for similarity search'}}, 'required': ['name', 'entityType', 'observations']}}}, 'required': ['entities']}, annotations=None),
 Tool(name='search_nodes', description='Search for entities and their relations using text or vector similarity', inputSchema={'type': 'object', 'properties': {'query': {'oneOf': [{'type': 'string', 'description': 'Text search query'}, {'type': 'array', 'items': {'type': 'number'}, 'description': 'Vector for similarity search'}]}}, 'required': ['query']}, annotations=None),
 Tool(name='read_graph', description='Get 

In [22]:
instructions = "Utilizas tus herramientas de entidad como una memoria persistente para almacenar y recordar información sobre tus conversaciones en un fichero .db."
request = "Me llamo Juan Gabriel. Soy matemático, especializado en ingeniería de LLM. Estoy enseñando un curso sobre Agentes de IA, incluyendo el increíble protocolo MCP. \
MCP es un protocolo para conectar agentes con herramientas, recursos y plantillas de prompts, y facilita la integración de agentes de IA con capacidades."
model = "gpt-4.1-mini"

In [23]:
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

Hola Juan Gabriel, es un placer conocerte. He registrado que eres matemático especializado en ingeniería de LLM y que estás enseñando un curso sobre Agentes de IA que incluye el protocolo MCP. Además, he guardado que MCP es un protocolo para conectar agentes con herramientas, recursos y plantillas de prompts, facilitando la integración de agentes de IA con capacidades. ¿En qué te puedo ayudar hoy respecto a tu curso o sobre MCP?

In [27]:
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, "Me llamo Juan Gabriel. ¿Qué sabes sobre mí? Utiliza las herramientas disponibles para encontrar información sobre mi.")
    display(Markdown(result.final_output))

Sé que eres Juan Gabriel, un matemático especializado en ingeniería de modelos de lenguaje grande (LLM). También eres profesor de un curso sobre Agentes de IA, en el que se incluye la enseñanza del protocolo MCP. ¿Quieres que te cuente más o que registre alguna información adicional sobre ti?

### Comprueba la traza:

https://platform.openai.com/traces

### El segundo tipo de servidor MCP: se ejecuta localmente, llama a un servicio web

### Brave Search - disculpa, ¡esto necesitará otra clave de API! Pero es gratis de nuevo.

https://brave.com/search/api/

# Crea tu cuenta y pon tu clave en el archivo .env con el nombre `BRAVE_API_KEY`

In [29]:
env = {"BRAVE_API_KEY": os.getenv("BRAVE_API_KEY")}
params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-brave-search"], "env": env}

async with MCPServerStdio(params=params) as server:
    mcp_tools = await server.list_tools()

mcp_tools

[Tool(name='brave_web_search', description='Performs a web search using the Brave Search API, ideal for general queries, news, articles, and online content. Use this for broad information gathering, recent events, or when you need diverse web sources. Supports pagination, content filtering, and freshness controls. Maximum 20 results per request, with offset for pagination. ', inputSchema={'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'Search query (max 400 chars, 50 words)'}, 'count': {'type': 'number', 'description': 'Number of results (1-20, default 10)', 'default': 10}, 'offset': {'type': 'number', 'description': 'Pagination offset (max 9, default 0)', 'default': 0}}, 'required': ['query']}, annotations=None),
 Tool(name='brave_local_search', description="Searches for local businesses and places using Brave's Local Search API. Best for queries related to physical locations, businesses, restaurants, services, etc. Returns detailed information including:\

In [30]:
instructions = "Puedes buscar información en la web y resumir brevemente las conclusiones."
request = f"Por favor, investiga las últimas noticias sobre el precio de las acciones de Amazon y resume brevemente su perspectiva. \
Para contexto, la fecha actual es {datetime.now().strftime('%Y-%m-%d')}"
model = "gpt-4o-mini"

In [31]:
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

Aquí tienes un resumen de las últimas noticias sobre el precio de las acciones de Amazon (AMZN) al 17 de julio de 2025:

1. **Cotización Actual**: El precio de las acciones de Amazon se encuentra en **$226.35** desde el 16 de julio de 2025.

2. **Perspectivas de Precio**: Se pronostica que el valor de las acciones de Amazon podría alcanzar **$244.38** para finales de 2025, con una proyección a largo plazo de **$294.75** para finales de 2029.

3. **Rango de Cotización**: Para el año 2025, los analistas estiman que el precio de las acciones oscilará entre **$151.78** y **$181.94**.

Estos datos sugieren una expectativa de crecimiento a mediano y largo plazo, con fluctuaciones en el corto plazo.

### Y como siempre, comprobamos la traza en:

https://platform.openai.com/traces

## Y ahora el tercer tipo: ejecución remota

En realidad, es bastante difícil encontrar un "servidor MCP remoto", también conocido como "servidor MCP alojado" o "servidor MCP gestionado".

No es un modelo común para usar o compartir servidores MCP, y no existe una forma estándar de descubrir servidores MCP remotos.

Anthropic enumera algunos servidores MCP remotos, pero estos son para aplicaciones de pago con usuarios empresariales:

https://docs.anthropic.com/en/docs/agents-and-tools/remote-mcp-servers

CloudFlare tiene herramientas para que puedas crear y desplegar tus propios servidores MCP remotos, pero esto no parece ser una práctica común:

https://developers.cloudflare.com/agents/guides/remote-mcp-server/


# Y de vuelta al segundo tipo: el servidor MCP de Polygon.io

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">¡¡POR FAVOR; LEE!!</h2>
             <span style="color:#ff7800;">Este servicio de datos financieros de mercado tiene tanto un plan GRATUITO como un plan DE PAGO, y podemos usar cualquiera de los dos según tu preferencia.
            </span>
        </td>
    </tr>
</table>

## NUEVA SECCIÓN: Introducción a polygon.io

Polygon.io es un proveedor de datos financieros muy popular. Tiene un plan gratuito y un plan de pago. ¡Y también tiene un servidor MCP!

Primero, infórmate sobre polygon.io en su excelente sitio web, incluyendo su sección de precios:

https://polygon.io

### Polygon.io Parte 1: Servicio gratuito de Polygon.io (el de pago será totalmente opcional, por supuesto)

1. Por favor, regístrate en polygon.io (arriba a la derecha)
2. Una vez dentro, selecciona "Keys" en la navegación de la izquierda
3. Pulsa el botón azul "New Key"
4. Copia el nombre de la clave
5. Edita tu archivo .env y añade la línea:

`POLYGON_API_KEY=xxxx`

In [32]:
load_dotenv(override=True)
polygon_api_key = os.getenv("POLYGON_API_KEY")
if not polygon_api_key:
    print("POLYGON_API_KEY is not set")

In [33]:
from polygon import RESTClient
client = RESTClient(polygon_api_key)
client.get_previous_close_agg("AAPL")[0]

PreviousCloseAgg(ticker='AAPL', close=210.16, high=212.4, low=208.64, open=210.295, timestamp=1752696000000, volume=47490532.0, vwap=210.3633)

### Envuelto en un módulo de Python que almacena en caché los precios de cierre

He creado un módulo de Python llamado `market.py` que utiliza esta API para consultar los precios de las acciones.

Pero la API gratuita tiene bastantes limitaciones de uso, así que he sido un poco astuto: cuando pides el precio de una acción, esta función recupera todo el mercado de acciones al cierre del día y lo almacena en caché en nuestra base de datos.


In [34]:
from market import get_share_price
get_share_price("AAPL")

210.16

In [35]:
# no rate limiting concerns!

for i in range(1000):
    get_share_price("AAPL")
get_share_price("AAPL")

210.16

### Y he convertido esto en un servidor MCP

Igual que hicimos con accounts.py; mira `market_server.py`

In [36]:
params = {"command": "uv", "args": ["run", "market_server.py"]}
async with MCPServerStdio(params=params) as server:
    mcp_tools = await server.list_tools()
mcp_tools

[Tool(name='lookup_share_price', description='Esta herramienta proporciona el precio actual del símbolo de acción dado.\n\n    Argumentos:\n        symbol: el símbolo de la acción\n    ', inputSchema={'properties': {'symbol': {'title': 'Symbol', 'type': 'string'}}, 'required': ['symbol'], 'title': 'lookup_share_priceArguments', 'type': 'object'}, annotations=None)]

### ¡Vamos a probarlo!

#Esperemos que gpt-4o-mini sea lo suficientemente inteligente como para saber que el símbolo de Apple es AAPL

In [37]:
instructions = "Respondes preguntas sobre el mercado de valores."
request = "¿Cuál es el precio de la acción de Apple?"
model = "gpt-4.1-mini"

async with MCPServerStdio(params=params) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

El precio actual de la acción de Apple (AAPL) es de 210.16 dólares. ¿Quieres saber algo más sobre esta acción o alguna otra?

## Polygon.io Parte 2: Plan de Pago - ¡Totalmente Opcional!
 
Si te interesa, puedes suscribirte al plan mensual para obtener datos de mercado más actualizados y llamadas ilimitadas a la API.

Si decides hacerlo, también tiene sentido usar el servidor MCP completo que Polygon.io ha lanzado, para aprovechar toda su funcionalidad.


In [ ]:

params = {"command": "uvx",
          "args": ["--from", "git+https://github.com/polygon-io/mcp_polygon@master", "mcp_polygon"],
          "env": {"POLYGON_API_KEY": polygon_api_key}
          }
async with MCPServerStdio(params=params) as server:
    mcp_tools = await server.list_tools()
mcp_tools


### ¡Vaya, eso son muchas herramientas!

¡Vamos a probarlas! Esperemos que la gran cantidad de herramientas no abrume a gpt-4o-mini.

Con el plan mensual de $29, no tenemos acceso a algunas de las APIs, así que he tenido que especificar cuáles se pueden usar.
 
Si te has suscrito a un plan más grande, siéntete libre de quitar mi restricción extra.

In [ ]:
instructions = "Respondes preguntas sobre el mercado de valores."
request = "¿Cuál es el precio de la acción de Apple? Usa tu herramienta get_snapshot_ticker para obtener el precio más reciente."
model = "gpt-4.1-mini"

async with MCPServerStdio(params=params) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

## Configurando tu archivo .env

Si decides suscribirte a un plan de pago, por favor añade esto a tu archivo .env para indicarlo:

`POLYGON_PLAN=paid`

Y si decides ir a por la API en tiempo real, entonces añade lo siguiente:

`POLYGON_PLAN=realtime`

In [43]:
load_dotenv(override=True)

polygon_plan = os.getenv("POLYGON_PLAN")
is_paid_polygon = polygon_plan == "paid"
is_realtime_polygon = polygon_plan == "realtime"

if is_paid_polygon:
    print("Has elegido suscribirte al plan de pago de Polygon, así que el código consultará precios con un retraso de 15 minutos")
elif is_realtime_polygon:
    print("¡Vaya! Has elegido suscribirte al plan en tiempo real de Polygon, así que el código consultará precios en tiempo real")
else:
    print("Según tu archivo .env, has elegido suscribirte al plan gratuito de Polygon, así que el código consultará precios de cierre diario (EOD)")

Según tu archivo .env, has elegido suscribirte al plan gratuito de Polygon, así que el código consultará precios de cierre diario (EOD)


## ¡Y eso es todo por hoy!

He eliminado la parte de este laboratorio que usaba el servidor MCP "Financial Datasets", porque es inferior: más caro y con menos APIs.

Dr esta manera, usamos el mismo proveedor tanto para las APIs gratuitas como para las de pago.

Pero si quieres ver el código, solo tienes que mirar en el historial de git una versión anterior.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Ejercicios</h2>
#             <span style="color:#ff7800;">Explora los marketplaces de servidores MCP e integra el tuyo propio, utilizando los 3 enfoques.
            </span>
        </td>
    </tr>
</table>